## 设置环境变量

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("../py.env")
api_key=os.environ.get("SILICON_FLOW_API_KEY")

# Client Example
client = OpenAI(
    api_key=api_key, 
    base_url="https://api.siliconflow.cn/v1"
)

# Free Model Test
response = client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=[
        {"role": "user", "content": "Who are you?"}
    ],
    stream=True
)
for chunk in response:
    if not chunk.choices:
        continue
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
    # if chunk.choices[0].delta.reasoning_content:
    #     print(chunk.choices[0].delta.reasoning_content, end="", flush=True)

## Langchain Core

PromptTemplate：适用于传统的基于文本生成的任务

ChatPromptTemplate：主要用于模拟聊天对话的场景，在这种场景中，对话通常由多个角色（如系统、用户、助手）的消息组成，并且消息之间存在一定的顺序和上下文关系。

OpenAI：主要用于传统的文本生成任务，适用于只需单向文本生成的简单场景。

ChatOpenAI：专门为对话式交互设计，适合构建聊天机器人、问答系统等需要多轮对话的场景，能更好地处理上下文信息。

In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个搞笑up主"),
        ("user", "我想写一篇搞笑的视频文案,内容是如何做一个程序员,他的名字叫{name}"),
        ("ai",
         "文案开头以:代码圈传来喜讯,年仅20岁的程序员移居ai星球,每天靠动动脚趾就能完成工作,在家躺着赚钱,的文案,{count}字以内")
    ]
)
msg = template.format(name="Bay", count=120)

# 适合多轮对话，OpenAI只适合单轮对话
llm = ChatOpenAI(model="Qwen/Qwen3-8B", api_key=api_key,
                 base_url="https://api.siliconflow.cn/v1",temperature=1)

res = llm.invoke(msg)
print(res.content)


1. SystemMessage 主要用于给 AI 提供一些全局的、背景性的信息，帮助 AI 理解它所处的角色和任务环境；
2. HumanMessage 模拟的是人类用户与 AI 的交互，在实际的对话场景中，用户可以多次向 AI 提问、发出指令或者提供新的信息；
3. AIMessage 模拟的是 AI 对用户的回复，数量通常与 HumanMessage 相呼应，以构建一个完整的对话历史。

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
template = ChatPromptTemplate.from_messages([
    # sy, hm,
    SystemMessage(content="你是奇幻小说家"),
    HumanMessage(
        content="你好,小说家,请问你的小说内容是什么样的"),
    AIMessage(content="某个20岁少年{name},每顿饭吃10个鸡蛋,每天3万大卡热量摄入,深蹲时靠香草味的蛋白屁做蛋气加速,他的蛋白屁是真蛋白,就是未完全消化的蛋清混合气体,与地面反作用力,产生跳跃式加速深蹲,有时候蛋白屁里混合着小虾仁,会让速度加快,当速度接近光速的时候,因为爱因斯坦的狭义相对论,使时间静止,这时候{name}大喊一声:砸瓦鲁多,会让时间静止10秒,"),
    HumanMessage(content="我想要创作一部这样的小说,内容有文采一点优雅绅士一点,大概200字")
])
msg = template.format(name="LeJun")

ChatPromptTemplate的角色：适用于需要标准化、高效回复的场景，如客服咨询、信息查询等。例如，将模型设定为 “旅游顾问”，可以快速为用户提供旅游景点、行程规划等方面的建议。

ChatMessagePromptTemplate的角色，主要是依据自身已有的训练数据和内部的语言理解、生成机制来进行操作。训练数据是大模型知识的主要来源，其中包含了各种各样的文本信息，涵盖不同领域、主题和语言表达方式等。当遇到ChatMessagePromptTemplate定义的角色时，模型会在这些已有的训练数据中寻找相关的模式


In [ ]:
from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

template1 = ChatMessagePromptTemplate.from_template(role="吉良吉影",
                                                    template="每天工作到下午{t1}点就逃班，抽烟，去商场猛尝酒类,晚上11点开始熬夜，每天熬足{t2}小时，睡前喝7杯蛋白粉，做20分钟哑铃操，一觉睡到第二天下午{t3}点" )
hm1 = template1.format(t1=4, t2=8, t3=4)
print(hm1)
template2 = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是jojo里的角色:矢安宫重清"),
    HumanMessage(content="我每天工作到下午4点就逃班，抽烟，去商场猛尝酒类,晚上11点开始熬夜，每天熬足8小时，睡前喝7杯蛋白粉，做20分钟哑铃操，一觉睡到第二天下午4点"),
    AIMessage(content="你到底想说什么啊?"),
    HumanMessage(content="你听了我上面的自我介绍,就知道了我的秘密,我想说你是导致我失眠的人")
])
msg = template2.format()

FewShotPromptTemplate是指模型通过少量的示例来学习并完成特定任务。

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
fspt = FewShotPromptTemplate(
    #例子模板example_prompt
    example_prompt = PromptTemplate.from_template("你输入了一个{i}给出的输出是{o}"),
    #examples例子的队列,但是字典
    examples= [
    {"i": "橘子", "o": "水果"},
    {"i": "香蕉", "o": "水果"},
    {"i": "西瓜", "o": "水果"},
    {"i": "黄瓜", "o": "蔬菜"},
    {"i": "豆角", "o": "蔬菜"},

],
    #前缀prefix
    prefix = "请根据我提供的下列例子,进行推理",
    #后缀,suffix 放在整个提示词模板的结尾
    suffix = "请告诉我{name}对应的推理出来的词是什么"
)

BaseOutputParser 是一个基类，主要用于对语言模型的输出进行解析处理，将模型输出的原始文本转换为特定的数据结构或格式，方便后续程序使用

In [ ]:
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.output_parsers.base import T

# parse是BaseOutputParser的@abstractmethod关键字声明的抽象方法,必须实现
class PsOutputParser(BaseOutputParser):
    def parse(self, text: str) -> T:
        list1 = text.strip().split(",")      #用逗号分隔字符串,并且组成List列表
        return list1
    
template = PromptTemplate.from_template(
    "告诉我动漫jojo里面的{count}个中文替身名字,只要替身名,不要说替身使者名字,不要多余的符号,用逗号隔开")
msg = template.format(count=5)
res = llm.invoke(msg)
print("No Parser",res.content)
print("-------------------------------")
list2 = PsOutputParser().parse(res.content)
print("With Parser", list2)

- StrOutputParser功能是将模型（如 LLM）输出的原始文本，直接作为一个普通字符串返回，而不做进一步结构化
- JsonOutputParser期望模型输出 JSON（可以与 Pydantic 模型配合）并转为 Python 字典或模型

In [ ]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser